# Interface between Copper (001) and Cristobalite (001)

## Introduction

This notebook demonstrates the creation of an interface between two 3D materials.

Following the manuscript:
    > **Shan, T.-R., Devine, B. D., Phillpot, S. R., & Sinnott, S. B.** 
    > "Molecular dynamics study of the adhesion of Cu/SiO2interfaces using a variable-charge interatomic potential."
    > Physical Review B, 83(11). 
    > [DOI: 10.1103/PhysRevB.83.115327](https://doi.org/10.1103/PhysRevB.83.115327) 



Replicating the materials from the manuscript, FIG. 1. (a):

<img src='https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/interfaces/interface_3d_3d_copper_cristobalite/0-figure-from-manuscript.webp' width='600'/>


## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior 

In [ ]:
FILM_MILLER_INDICES = (0, 0, 1)
FILM_THICKNESS = 3  # in atomic layers
FILM_TERMINATION_FORMULA = None  # if None, the first termination will be used
FILM_VACUUM = 0.0  # in angstroms

SUBSTRATE_MILLER_INDICES = (0, 0, 1)
SUBSTRATE_THICKNESS = 3  # in atomic layers
SUBSTRATE_TERMINATION_FORMULA = None  # if None, the first termination will be used
SUBSTRATE_VACUUM = 0.0  # in angstroms

INTERFACE_DISTANCE = 2.4  # Gap between substrate and film, in Angstrom
INTERFACE_VACUUM = 18.0  # Vacuum over film, in Angstrom

# Whether to convert materials to conventional cells before creating slabs.
USE_CONVENTIONAL_CELL = True

# Maximum area for the superlattice search algorithm (the final interface area will be smaller)
MAX_AREA = 150  # in Angstrom^2
# Additional fine-tuning parameters (increase values to get more strained matches):
MAX_AREA_TOLERANCE = 0.09  # in Angstrom^2
MAX_LENGTH_TOLERANCE = 0.05
MAX_ANGLE_TOLERANCE = 0.02

# Whether to reduce the resulting interface cell to the primitive cell after the interface creation.
REDUCE_RESULT_CELL_TO_PRIMITIVE = False

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples|create_interface_with_min_strain_zsl.ipynb")

### 1.3. Get input materials and assign `substrate` and `film`
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

film = Material.create(Materials.get_by_name_first_match("Copper"))
substrate = Material.create(Materials.get_by_name_first_match("Cristobalite"))

### 1.4. Preview Substrate and Film

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

## 2. Configure slabs for interface

### 2.1. Get possible terminations for the slabs

In [ ]:
from mat3ra.made.tools.helpers import get_slab_terminations

film_slab_terminations = get_slab_terminations(material=film, miller_indices=FILM_MILLER_INDICES)
substrate_slab_terminations = get_slab_terminations(material=substrate, miller_indices=SUBSTRATE_MILLER_INDICES)
print("Film slab terminations:", film_slab_terminations)
print("Substrate slab terminations:", substrate_slab_terminations)


### 2.2. Visualize slabs for all possible terminations

In [ ]:
from mat3ra.made.tools.helpers import create_slab, select_slab_termination

film_slabs = [create_slab(film, miller_indices=FILM_MILLER_INDICES, termination_top=termination, vacuum=0) for termination
              in
              film_slab_terminations]

substrate_slabs = [create_slab(substrate, miller_indices=SUBSTRATE_MILLER_INDICES, termination_top=termination, vacuum=0)
                   for termination in
                   substrate_slab_terminations]

film_slabs_with_titles = [{"material": slab, "title": str(termination)} for slab, termination in
                          zip(film_slabs, film_slab_terminations)]
substrate_slabs_with_titles = [{"material": slab, "title": str(termination)} for slab, termination in
                               zip(substrate_slabs, substrate_slab_terminations)]

visualize(film_slabs_with_titles, repetitions=[3, 3, 1], rotation="-90x")
visualize(substrate_slabs_with_titles, repetitions=[3, 3, 1], rotation="-90x")


### 2.3. Select terminations for the Slabs

In [ ]:
film_termination = select_slab_termination(film_slab_terminations, FILM_TERMINATION_FORMULA)
substrate_termination = select_slab_termination(substrate_slab_terminations, SUBSTRATE_TERMINATION_FORMULA)

### 2.4. Create Substrate and Film Slabs
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane.
Define the substrate slab cell that will be used as a base for the interface and the film slab cell that will be placed on top of the substrate slab.

In [ ]:
from mat3ra.made.tools.build.pristine_structures.two_dimensional.slab import SlabConfiguration, SlabBuilder

substrate_slab_config = SlabConfiguration.from_parameters(
    material_or_dict=substrate,
    miller_indices=SUBSTRATE_MILLER_INDICES,
    number_of_layers=SUBSTRATE_THICKNESS,
    vacuum=0.0,
    termination_top_formula=SUBSTRATE_TERMINATION_FORMULA,
    use_conventional_cell=USE_CONVENTIONAL_CELL
)

film_slab_config = SlabConfiguration.from_parameters(
    material_or_dict=film,
    miller_indices=FILM_MILLER_INDICES,
    number_of_layers=FILM_THICKNESS,
    vacuum=0.0,
    termination_bottom_formula=FILM_TERMINATION_FORMULA, # film termination is on the bottom
    use_conventional_cell=USE_CONVENTIONAL_CELL
)

substrate_slab = SlabBuilder().get_material(substrate_slab_config)
film_slab = SlabBuilder().get_material(film_slab_config)

## 3. Analyze possible interfaces with ZSL Analyzer
### 3.1. Initialize ZSL Analyzer
The search algorithm for supercells matching can be tuned by setting its parameters directly, otherwise the default values are used.


In [ ]:
from mat3ra.made.tools.analyze.interface import ZSLInterfaceAnalyzer

zsl_analyzer = ZSLInterfaceAnalyzer(
    substrate_slab_configuration=substrate_slab_config,
    film_slab_configuration=film_slab_config,
    max_area=MAX_AREA,
    max_area_ratio_tol=MAX_AREA_TOLERANCE,
    max_length_tol=MAX_LENGTH_TOLERANCE,
    max_angle_tol=MAX_ANGLE_TOLERANCE,
    reduce_result_cell=False  # Reduces supercell matrices in analyzer
)


### 3.2. Generate matches with strain analyzer
Matches are sorted by size and strain.


In [ ]:
matches = zsl_analyzer.zsl_match_holders


### 3.3. Plot matches by area and strain


In [ ]:
from utils.plot import plot_strain_vs_area

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}

plot_strain_vs_area(matches, PLOT_SETTINGS)


### 3.4. Select the interface

Select the index for the interface with the lowest strain and the smallest area.


In [ ]:
selected_index = 0

from mat3ra.made.tools.helpers import create_interface_zsl_between_slabs as create_zsl_interface_between_slabs

interface = create_zsl_interface_between_slabs(
    substrate_slab=substrate_slab,
    film_slab=film_slab,
    gap=INTERFACE_DISTANCE,
    vacuum=INTERFACE_VACUUM,
    match_id=selected_index,
    max_area=MAX_AREA,
    max_area_ratio_tol=MAX_AREA_TOLERANCE,
    max_length_tol=MAX_LENGTH_TOLERANCE,
    max_angle_tol=MAX_ANGLE_TOLERANCE,
    reduce_result_cell_to_primitive=REDUCE_RESULT_CELL_TO_PRIMITIVE,
)


## 4. Preview the selected interface


In [ ]:
visualize(interface, repetitions=[1, 1, 1])
visualize(interface, repetitions=[1, 1, 1], rotation="-90x")


## 5. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import set_materials, download_content_to_file

set_materials(interface)
download_content_to_file(interface.to_json(), f"{interface.name}.json")